# Fake News Classifier using LSTM

In [64]:
import pandas as pd

In [65]:
# Reading the input file
df = pd.read_csv('train.csv')

In [66]:
# Print top 5 rows
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


# Text Preprocessing

In [67]:
# Check  for missing values
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
# Dropping the missing values and reseting the index
df = df.dropna()
df = df.reset_index(drop=True)
df.shape

(18285, 5)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18285 entries, 0 to 18284
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 714.4+ KB


In [8]:
# Check for class imbalance
df['label'].value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [9]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [10]:
# importing librarires for text preprocessing
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps = PorterStemmer()

In [11]:
# Preprocessing text data 
corpus = []
for i in range(0, len(df['title'])):
    # regular to keep only alphabets
    title = re.sub('[^a-zA-Z]', ' ', df['title'][i])
    # To convert alphabets into lower case
    title = title.lower()
    title = title.split()
    # Stemming of words
    title = [ps.stem(word) for word in title if not word in stopwords.words('english')]
    title = ' '.join(title)
    corpus.append(title)

In [71]:
corpus[0:11]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo']

# One-Hot Representation

 Now we will convert this corpus into one hot representation.
 In One hot, index will be assigned based on vocabulary size.
 one hot will generate index of words based on vocabulary size.

In [14]:
vocabulary_size = 5000

In [15]:
onehot_repr = [one_hot(words, vocabulary_size) for words in corpus]
onehot_repr[0:11]

[[3704, 773, 741, 2846, 4459, 1884, 2631, 2701, 4201, 276],
 [3055, 2396, 2908, 886, 3503, 539, 4982],
 [2374, 1183, 4151, 3575],
 [2684, 4461, 301, 707, 818, 1194],
 [1747, 3503, 1330, 1526, 343, 4820, 3503, 2628, 4336, 3985],
 [2358,
  3120,
  2143,
  2406,
  1099,
  4364,
  2490,
  2314,
  4779,
  208,
  4942,
  1869,
  3526,
  1330,
  4982],
 [1426, 4758, 3097, 240, 993, 310, 2242, 795, 1282, 701, 4172],
 [1356, 539, 1260, 3252, 1302, 3925, 4364, 2560, 1282, 701, 4172],
 [4324, 2862, 4831, 4127, 4615, 2620, 1045, 2055, 4364, 4719],
 [712, 2681, 3868, 3992, 2687, 1465, 1117, 2755],
 [9, 312, 4689, 3574, 4736, 46, 1220, 1010, 2345, 4822, 3322]]

# Embedding Representation

After one hot representation, all the senntences length are different. Before Passing it to the emdedding layer we need to make each sentence are in the fixed length. It can be done using pad_sequences. 

In [16]:
sentence_length = 20

In [17]:
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sentence_length)
embedded_docs[0:11]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3704,
         773,  741, 2846, 4459, 1884, 2631, 2701, 4201,  276],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 3055, 2396, 2908,  886, 3503,  539, 4982],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 2374, 1183, 4151, 3575],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 2684, 4461,  301,  707,  818, 1194],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1747,
        3503, 1330, 1526,  343, 4820, 3503, 2628, 4336, 3985],
       [   0,    0,    0,    0,    0, 2358, 3120, 2143, 2406, 1099, 4364,
        2490, 2314, 4779,  208, 4942, 1869, 3526, 1330, 4982],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0, 1426, 4758,
        3097,  240,  993,  310, 2242,  795, 1282,  701, 4172],
       [   0,    0,    0,    0,    0,    

In [18]:
embedding_vector_features = 40

# Train/Test Split

In [19]:
import numpy as np
X = np.array(embedded_docs)
y = np.array(df['label'])
print(X.shape, y.shape)

(18285, 20) (18285,)


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state =111)

In [21]:
X_train.shape

(12799, 20)

# Function for Model Evaluation

In [22]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
def modelEvaluation(predictions):
    print("\nAccuracy on test set: {:4f}".format(accuracy_score(y_test,predictions)))
    print("\nClassification report : \n",classification_report(y_test,predictions))
    print("\nConfusion matrix : \n", confusion_matrix(y_test, predictions))

In [68]:
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.layers import LSTM
from keras.optimizers import Adam

# Creating Model 

In [23]:
def create_model (learning_rate):
    model = Sequential()
    model.add(Embedding(vocabulary_size, embedding_vector_features, input_length = sentence_length))
    model.add(Dropout(0.3))
    model.add(LSTM(100))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation = 'sigmoid'))
    monitor_val_acc = EarlyStopping(monitor='loss', patience=3)
    opt = Adam(learning_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics=['accuracy'])
    return model

In [24]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn = create_model, verbose=1)

In [25]:
from sklearn.model_selection import GridSearchCV
params = {'learning_rate' : [0.1, 0.001, 0.01], 'batch_size' :[32, 64], 'epochs' : [10,15]}
grid_search = GridSearchCV(estimator = model, param_grid = params, cv=3 )
results = grid_search.fit(X_train, y_train)
print(results.best_params_)

Epoch 1/10
267/267 [==============================] - 7s 27ms/step - loss: 0.3322 - accuracy: 0.8733
Epoch 2/10
267/267 [==============================] - 7s 27ms/step - loss: 0.3558 - accuracy: 0.8697
Epoch 3/10
267/267 [==============================] - 7s 26ms/step - loss: 0.3559 - accuracy: 0.8650
Epoch 4/10
267/267 [==============================] - 6s 24ms/step - loss: 0.3608 - accuracy: 0.8635
Epoch 5/10
267/267 [==============================] - 7s 28ms/step - loss: 0.3498 - accuracy: 0.8686
Epoch 6/10
267/267 [==============================] - 7s 28ms/step - loss: 0.3601 - accuracy: 0.8694
Epoch 7/10
267/267 [==============================] - 7s 28ms/step - loss: 0.3415 - accuracy: 0.8692
Epoch 8/10
267/267 [==============================] - 7s 27ms/step - loss: 0.3468 - accuracy: 0.8687
Epoch 9/10
267/267 [==============================] - 7s 28ms/step - loss: 0.3615 - accuracy: 0.8671
Epoch 10/10
134/134 [==============================] - 1s 9ms/step - loss: 0.3136 - accurac

267/267 [==============================] - 5s 17ms/step - loss: 0.0242 - accuracy: 0.9916
Epoch 6/10
267/267 [==============================] - 4s 15ms/step - loss: 0.0197 - accuracy: 0.9932
Epoch 7/10
267/267 [==============================] - 5s 20ms/step - loss: 0.0168 - accuracy: 0.9952
Epoch 8/10
267/267 [==============================] - 5s 17ms/step - loss: 0.0165 - accuracy: 0.9940
Epoch 9/10
267/267 [==============================] - 4s 15ms/step - loss: 0.0159 - accuracy: 0.9946
Epoch 10/10
134/134 [==============================] - 1s 4ms/step - loss: 0.4282 - accuracy: 0.9142
Epoch 1/10
267/267 [==============================] - 5s 18ms/step - loss: 0.2472 - accuracy: 0.8935
Epoch 2/10
267/267 [==============================] - 4s 16ms/step - loss: 0.1346 - accuracy: 0.9474
Epoch 3/10
267/267 [==============================] - 5s 19ms/step - loss: 0.0719 - accuracy: 0.9734
Epoch 4/10
267/267 [==============================] - 5s 18ms/step - loss: 0.0451 - accuracy: 0.9841
E

134/134 [==============================] - 0s 4ms/step - loss: 0.6123 - accuracy: 0.9098
Epoch 1/15
267/267 [==============================] - 4s 15ms/step - loss: 0.3237 - accuracy: 0.8493
Epoch 2/15
267/267 [==============================] - 4s 15ms/step - loss: 0.1488 - accuracy: 0.9395
Epoch 3/15
267/267 [==============================] - 4s 16ms/step - loss: 0.1000 - accuracy: 0.9625
Epoch 4/15
267/267 [==============================] - 4s 16ms/step - loss: 0.0733 - accuracy: 0.9723
Epoch 5/15
267/267 [==============================] - 5s 17ms/step - loss: 0.0474 - accuracy: 0.9841
Epoch 6/15
267/267 [==============================] - 5s 18ms/step - loss: 0.0342 - accuracy: 0.9891
Epoch 7/15
267/267 [==============================] - 5s 18ms/step - loss: 0.0226 - accuracy: 0.9921
Epoch 8/15
267/267 [==============================] - 5s 18ms/step - loss: 0.0170 - accuracy: 0.9944
Epoch 9/15
267/267 [==============================] - 7s 25ms/step - loss: 0.0165 - accuracy: 0.9947
Ep

134/134 [==============================] - 3s 21ms/step - loss: 0.3358 - accuracy: 0.8687
Epoch 2/10
134/134 [==============================] - 3s 24ms/step - loss: 0.2769 - accuracy: 0.8932
Epoch 3/10
134/134 [==============================] - 3s 22ms/step - loss: 0.2664 - accuracy: 0.8962
Epoch 4/10
134/134 [==============================] - 3s 20ms/step - loss: 0.2682 - accuracy: 0.8971
Epoch 5/10
134/134 [==============================] - 3s 19ms/step - loss: 0.2694 - accuracy: 0.8944
Epoch 6/10
134/134 [==============================] - 3s 19ms/step - loss: 0.2602 - accuracy: 0.8966
Epoch 7/10
134/134 [==============================] - 3s 19ms/step - loss: 0.2655 - accuracy: 0.8947
Epoch 8/10
134/134 [==============================] - 3s 19ms/step - loss: 0.2704 - accuracy: 0.8959
Epoch 9/10
134/134 [==============================] - 3s 19ms/step - loss: 0.2768 - accuracy: 0.8937
Epoch 10/10
67/67 [==============================] - 0s 6ms/step - loss: 0.3078 - accuracy: 0.8882
Epo

134/134 [==============================] - 3s 21ms/step - loss: 0.0280 - accuracy: 0.9906
Epoch 6/10
134/134 [==============================] - 3s 21ms/step - loss: 0.0195 - accuracy: 0.9945
Epoch 7/10
134/134 [==============================] - 3s 21ms/step - loss: 0.0185 - accuracy: 0.9933
Epoch 8/10
134/134 [==============================] - 3s 21ms/step - loss: 0.0100 - accuracy: 0.9972
Epoch 9/10
134/134 [==============================] - 3s 21ms/step - loss: 0.0096 - accuracy: 0.9971
Epoch 10/10
67/67 [==============================] - 0s 7ms/step - loss: 0.4805 - accuracy: 0.9170
Epoch 1/10
134/134 [==============================] - 3s 22ms/step - loss: 0.2557 - accuracy: 0.8867
Epoch 2/10
134/134 [==============================] - 3s 22ms/step - loss: 0.1282 - accuracy: 0.9522
Epoch 3/10
134/134 [==============================] - 3s 22ms/step - loss: 0.0728 - accuracy: 0.9716
Epoch 4/10
134/134 [==============================] - 3s 22ms/step - loss: 0.0361 - accuracy: 0.9870
Epo

67/67 [==============================] - 0s 7ms/step - loss: 0.5770 - accuracy: 0.9138
Epoch 1/15
134/134 [==============================] - 3s 21ms/step - loss: 0.4022 - accuracy: 0.8133
Epoch 2/15
134/134 [==============================] - 3s 21ms/step - loss: 0.1677 - accuracy: 0.9348
Epoch 3/15
134/134 [==============================] - 3s 21ms/step - loss: 0.1127 - accuracy: 0.9590
Epoch 4/15
134/134 [==============================] - 3s 21ms/step - loss: 0.0816 - accuracy: 0.9730
Epoch 5/15
134/134 [==============================] - 3s 21ms/step - loss: 0.0590 - accuracy: 0.9805
Epoch 6/15
134/134 [==============================] - 3s 21ms/step - loss: 0.0429 - accuracy: 0.9846
Epoch 7/15
134/134 [==============================] - 3s 21ms/step - loss: 0.0278 - accuracy: 0.9911
Epoch 8/15
134/134 [==============================] - 3s 21ms/step - loss: 0.0178 - accuracy: 0.9940
Epoch 9/15
134/134 [==============================] - 3s 21ms/step - loss: 0.0161 - accuracy: 0.9948
Epoc

400/400 [==============================] - 6s 16ms/step - loss: 0.2246 - accuracy: 0.9016
Epoch 2/10
400/400 [==============================] - 6s 16ms/step - loss: 0.1295 - accuracy: 0.9516
Epoch 3/10
400/400 [==============================] - 6s 16ms/step - loss: 0.0739 - accuracy: 0.9737
Epoch 4/10
400/400 [==============================] - 6s 16ms/step - loss: 0.0468 - accuracy: 0.9821
Epoch 5/10
400/400 [==============================] - 6s 16ms/step - loss: 0.0331 - accuracy: 0.9877
Epoch 6/10
400/400 [==============================] - 7s 16ms/step - loss: 0.0254 - accuracy: 0.9915
Epoch 7/10
400/400 [==============================] - 6s 16ms/step - loss: 0.0180 - accuracy: 0.9934
Epoch 8/10
400/400 [==============================] - 6s 16ms/step - loss: 0.0235 - accuracy: 0.9918
Epoch 9/10
400/400 [==============================] - 7s 16ms/step - loss: 0.0161 - accuracy: 0.9955
Epoch 10/10
400/400 [==============================] - 6s 16ms/step - loss: 0.0151 - accuracy: 0.9953


# Prediction and Evaluation

In [69]:
y_pred = grid_search.predict(X_test)
prediction_lstm = (y_pred > 0.5)

172/172 [==============================] - 1s 4ms/step


In [70]:
modelEvaluation(prediction_lstm)


Accuracy on test set: 0.911411

Classification report : 
               precision    recall  f1-score   support

           0       0.92      0.92      0.92      3135
           1       0.89      0.90      0.90      2351

    accuracy                           0.91      5486
   macro avg       0.91      0.91      0.91      5486
weighted avg       0.91      0.91      0.91      5486


Confusion matrix : 
 [[2887  248]
 [ 238 2113]]
